In [4]:
import pandas as pd
from sqlalchemy import create_engine

DB_USER = "postgres"
DB_PASS = "Tom&Jerry2704"
DB_HOST = "localhost"
DB_PORT = 5432
DB_NAME = "music_analytics"

engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

df = pd.read_sql("""
    SELECT user_id, session_id, song_id, ts, played_ms, skipped
    FROM listens
    ORDER BY user_id, session_id, ts;
""", engine)

df.head()



,user_id,session_id,song_id,ts,played_ms,skipped
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,2018-07-15 10:30:00+00:00,180066,False
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,2018-07-15 10:30:00+00:00,236796,False
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,2018-07-15 10:30:00+00:00,231266,False
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,2018-07-15 10:30:00+00:00,169826,False
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_64f3743c-f624-46bb-a579-0f3f9a07a123,2018-07-15 10:30:00+00:00,210545,False


In [5]:
# Group listens by user_id + session_id
session_groups = df.groupby(["user_id", "session_id"])

# Compute session-level metrics
session_summary = session_groups.agg(
    session_start=("ts", "min"),
    session_end=("ts", "max"),
    session_duration_min=("ts", lambda x: (x.max() - x.min()).total_seconds() / 60),
    num_events=("song_id", "count"),
    num_skipped=("skipped", "sum"),
    skip_rate=("skipped", "mean"),
    avg_played_ms=("played_ms", "mean")
).reset_index()

session_summary.head()


,user_id,session_id,session_start,session_end,session_duration_min,num_events,num_skipped,skip_rate,avg_played_ms
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2018-07-15 10:30:00+00:00,2018-07-15 10:30:00+00:00,0.0,20,14,0.700000,200536.350000
1,0_0000a72b-09ac-412f-b452-9b9e79bded8f,0_0000a72b-09ac-412f-b452-9b9e79bded8f,2018-07-15 08:30:00+00:00,2018-07-15 09:30:00+00:00,60.0,17,13,0.764706,277617.764706
2,0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a,0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a,2018-07-15 13:30:00+00:00,2018-07-15 13:30:00+00:00,0.0,20,18,0.900000,237473.350000
3,0_00016a3d-9076-4f67-918f-f29e3ce160dc,0_00016a3d-9076-4f67-918f-f29e3ce160dc,2018-07-15 11:30:00+00:00,2018-07-15 11:30:00+00:00,0.0,20,16,0.800000,194780.450000
4,0_00018b58-deb8-4f98-ac5e-d7e01b346130,0_00018b58-deb8-4f98-ac5e-d7e01b346130,2018-07-15 07:30:00+00:00,2018-07-15 07:30:00+00:00,0.0,11,5,0.454545,197068.363636


In [7]:
# Load song metadata + audio features
songs = pd.read_sql("""
    SELECT song_id, release_year, duration_ms, tempo, valence, energy, danceability
    FROM songs;
""", engine)

songs.head()



,song_id,release_year,duration_ms,tempo,valence,energy,danceability
0,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,2018,180,134.025,0.152255,0.553465,0.653119
1,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,2018,236,130.038,0.337152,0.726828,0.877394
2,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,2018,231,145.028,0.373862,0.563009,0.679719
3,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,2018,169,111.982,0.649420,0.529484,0.864881
4,t_64f3743c-f624-46bb-a579-0f3f9a07a123,2018,210,147.031,0.652921,0.650057,0.857778


In [8]:
# Merge listens dataframe with song-level audio features
df_enriched = df.merge(
    songs,
    on="song_id",
    how="left"
)

df_enriched.head()


,user_id,session_id,song_id,ts,played_ms,skipped,release_year,duration_ms,tempo,valence,energy,danceability
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,2018-07-15 10:30:00+00:00,180066,False,2018,180,134.025,0.152255,0.553465,0.653119
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,2018-07-15 10:30:00+00:00,236796,False,2018,236,130.038,0.337152,0.726828,0.877394
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,2018-07-15 10:30:00+00:00,231266,False,2018,231,145.028,0.373862,0.563009,0.679719
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,2018-07-15 10:30:00+00:00,169826,False,2018,169,111.982,0.649420,0.529484,0.864881
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_64f3743c-f624-46bb-a579-0f3f9a07a123,2018-07-15 10:30:00+00:00,210545,False,2018,210,147.031,0.652921,0.650057,0.857778


In [9]:
session_enriched = df_enriched.groupby(["user_id", "session_id"]).agg(
    session_start=("ts", "min"),
    session_duration_min=("ts", lambda x: (x.max() - x.min()).total_seconds() / 60),
    num_events=("song_id", "count"),
    skip_rate=("skipped", "mean"),
    
    # musical features:
    avg_tempo=("tempo", "mean"),
    avg_valence=("valence", "mean"),
    avg_energy=("energy", "mean"),
    avg_danceability=("danceability", "mean")
).reset_index()

session_enriched.head()


,user_id,session_id,session_start,session_duration_min,num_events,skip_rate,avg_tempo,avg_valence,avg_energy,avg_danceability
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2018-07-15 10:30:00+00:00,0.0,20,0.700000,122.639850,0.400669,0.558843,0.725832
1,0_0000a72b-09ac-412f-b452-9b9e79bded8f,0_0000a72b-09ac-412f-b452-9b9e79bded8f,2018-07-15 08:30:00+00:00,60.0,17,0.764706,122.076059,0.537176,0.827561,0.477013
2,0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a,0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a,2018-07-15 13:30:00+00:00,0.0,20,0.900000,127.727800,0.467760,0.601848,0.747877
3,0_00016a3d-9076-4f67-918f-f29e3ce160dc,0_00016a3d-9076-4f67-918f-f29e3ce160dc,2018-07-15 11:30:00+00:00,0.0,20,0.800000,110.562900,0.757056,0.830732,0.661859
4,0_00018b58-deb8-4f98-ac5e-d7e01b346130,0_00018b58-deb8-4f98-ac5e-d7e01b346130,2018-07-15 07:30:00+00:00,0.0,11,0.454545,117.476636,0.530752,0.643925,0.656623


In [10]:
user_profiles = session_enriched.groupby("user_id").agg(
    num_sessions=("session_id", "count"),
    avg_session_duration=("session_duration_min", "mean"),
    avg_skip_rate=("skip_rate", "mean"),
    avg_tempo=("avg_tempo", "mean"),
    avg_valence=("avg_valence", "mean"),
    avg_energy=("avg_energy", "mean"),
    avg_danceability=("avg_danceability", "mean")
).reset_index()

user_profiles.head()


,user_id,num_sessions,avg_session_duration,avg_skip_rate,avg_tempo,avg_valence,avg_energy,avg_danceability
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,0.0,0.700000,122.639850,0.400669,0.558843,0.725832
1,0_0000a72b-09ac-412f-b452-9b9e79bded8f,1,60.0,0.764706,122.076059,0.537176,0.827561,0.477013
2,0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a,1,0.0,0.900000,127.727800,0.467760,0.601848,0.747877
3,0_00016a3d-9076-4f67-918f-f29e3ce160dc,1,0.0,0.800000,110.562900,0.757056,0.830732,0.661859
4,0_00018b58-deb8-4f98-ac5e-d7e01b346130,1,0.0,0.454545,117.476636,0.530752,0.643925,0.656623


In [11]:
user_profiles.shape

(10000, 8)

In [12]:
# Merge listen events with session-level features
df_ml = df_enriched.merge(
    session_enriched[[
        "user_id", "session_id", "session_duration_min",
        "num_events", "skip_rate", "avg_tempo", "avg_valence",
        "avg_energy", "avg_danceability"
    ]],
    on=["user_id", "session_id"],
    how="left"
)

# Then merge with user-level profiles
df_ml = df_ml.merge(
    user_profiles,
    on="user_id",
    how="left",
    suffixes=("", "_user")
)

df_ml.head()


,user_id,session_id,song_id,ts,played_ms,skipped,release_year,duration_ms,tempo,valence,...,avg_valence,avg_energy,avg_danceability,num_sessions,avg_session_duration,avg_skip_rate,avg_tempo_user,avg_valence_user,avg_energy_user,avg_danceability_user
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,2018-07-15 10:30:00+00:00,180066,False,2018,180,134.025,0.152255,...,0.400669,0.558843,0.725832,1,0.0,0.7,122.63985,0.400669,0.558843,0.725832
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,2018-07-15 10:30:00+00:00,236796,False,2018,236,130.038,0.337152,...,0.400669,0.558843,0.725832,1,0.0,0.7,122.63985,0.400669,0.558843,0.725832
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,2018-07-15 10:30:00+00:00,231266,False,2018,231,145.028,0.373862,...,0.400669,0.558843,0.725832,1,0.0,0.7,122.63985,0.400669,0.558843,0.725832
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,2018-07-15 10:30:00+00:00,169826,False,2018,169,111.982,0.649420,...,0.400669,0.558843,0.725832,1,0.0,0.7,122.63985,0.400669,0.558843,0.725832
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_64f3743c-f624-46bb-a579-0f3f9a07a123,2018-07-15 10:30:00+00:00,210545,False,2018,210,147.031,0.652921,...,0.400669,0.558843,0.725832,1,0.0,0.7,122.63985,0.400669,0.558843,0.725832


In [13]:
df_ml.shape

(166205, 26)

In [14]:
# Feature columns to use in simple skip prediction model
feature_cols = [
    "played_ms", "hour_of_day",
    "tempo", "valence", "energy", "danceability",
    "session_duration_min", "skip_rate", "num_events",
    "avg_skip_rate_user", "avg_session_duration_user"
]

# Make sure columns exist (some may be missing, so keep only present ones)
feature_cols = [c for c in feature_cols if c in df_ml.columns]

X = df_ml[feature_cols]
y = df_ml["skipped"].astype(int)

X.head(), y.head()


(   played_ms    tempo   valence    energy  danceability  session_duration_min  \
 0     180066  134.025  0.152255  0.553465      0.653119                   0.0   
 1     236796  130.038  0.337152  0.726828      0.877394                   0.0   
 2     231266  145.028  0.373862  0.563009      0.679719                   0.0   
 3     169826  111.982  0.649420  0.529484      0.864881                   0.0   
 4     210545  147.031  0.652921  0.650057      0.857778                   0.0   
 
    skip_rate  num_events  
 0        0.7          20  
 1        0.7          20  
 2        0.7          20  
 3        0.7          20  
 4        0.7          20  ,
 0    0
 1    0
 2    0
 3    0
 4    0
 Name: skipped, dtype: int64)

In [15]:
X.shape


(166205, 8)

In [17]:
# Cell: retrain logistic with scaling + more iterations + show coeffs
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np
import pandas as pd

# Use X, y from earlier scope (df_ml -> X, y)
# If X/y were overwritten, re-create them as before.

# 1) train-test split (same random_state for reproducibility)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2) scale features
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

# 3) train logistic with higher max_iter
model = LogisticRegression(max_iter=5000, solver='lbfgs')
model.fit(X_train_s, y_train)

# 4) predict + metrics
y_pred = model.predict(X_test_s)
y_prob = model.predict_proba(X_test_s)[:,1]
acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)

# 5) coefficients (map to feature names)
coefs = pd.Series(model.coef_[0], index=X.columns).sort_values(key=abs, ascending=False)

print(f"Accuracy: {acc:.4f}   AUC: {auc:.4f}")
print("\nTop coefficients (abs-sorted):")
print(coefs.head(12))


Accuracy: 0.7664   AUC: 0.8117

Top coefficients (abs-sorted):
skip_rate               1.268743
played_ms               0.115076
danceability           -0.038292
num_events              0.025262
energy                 -0.018573
valence                 0.015566
session_duration_min   -0.013321
tempo                   0.000353
dtype: float64


In [19]:
df[["user_id", "session_id"]].drop_duplicates().head(10)


,user_id,session_id
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,0_00006f66-33e5-4de7-a324-2d18e439fc1e
20,0_0000a72b-09ac-412f-b452-9b9e79bded8f,0_0000a72b-09ac-412f-b452-9b9e79bded8f
37,0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a,0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a
57,0_00016a3d-9076-4f67-918f-f29e3ce160dc,0_00016a3d-9076-4f67-918f-f29e3ce160dc
77,0_00018b58-deb8-4f98-ac5e-d7e01b346130,0_00018b58-deb8-4f98-ac5e-d7e01b346130
88,0_00027db1-ffe5-4b00-8eeb-8d8188451298,0_00027db1-ffe5-4b00-8eeb-8d8188451298
103,0_00029482-6542-4d3e-bb9e-ffc988f58bd6,0_00029482-6542-4d3e-bb9e-ffc988f58bd6
118,0_0002fbb5-29c2-4d54-9636-e6d15e77b58c,0_0002fbb5-29c2-4d54-9636-e6d15e77b58c
138,0_00030fe3-298d-4930-8534-fc5a9b82971b,0_00030fe3-298d-4930-8534-fc5a9b82971b
158,0_0003ac37-c70a-47a6-b9dc-b8b163826082,0_0003ac37-c70a-47a6-b9dc-b8b163826082


In [27]:
# --- Fixed test cell (no missing columns) ---
user_id = "0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a"
session_id = "0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a"

# Step 1: Get last 3 listens
last3 = df[(df.user_id==user_id) & (df.session_id==session_id)].sort_values("ts").tail(3)

if last3.empty:
    print("No rows found for that user/session — double-check the IDs.")
else:
    # Step 2: Merge with song features that actually exist
    last3_enriched = last3.merge(
        songs[["song_id","tempo","valence","energy","danceability","release_year","duration_ms"]],
        on="song_id",
        how="left"
    )
    
    print("Last 3 listens (with audio features):")
    display(last3_enriched[[
        "ts","song_id","played_ms","skipped",
        "tempo","valence","energy","danceability","release_year"
    ]])

    # Step 3: Build session embedding
    song_ids_last3 = list(last3_enriched["song_id"].values)
    print("\nLast-3 song IDs:", song_ids_last3)

    sess_vec = session_embedding_from_ids(song_ids_last3)
    if sess_vec is None:
        print("Could not build session embedding — missing features.")
    else:
        print("\nSession embedding (scaled feature vector):")
        print(sess_vec)

        # Step 4: Get recommendations
        recs = recommend_for_session(user_id, session_id, top_k=10, last_n=3)
        if recs is None or recs.empty:
            print("No recommendations returned.")
        else:
            # Merge recs with song metadata
            out = recs.merge(songs, on="song_id", how="left")
            print("\nTop 10 recommendations:")
            display(out[[
                "song_id","sim","tempo","valence","energy","danceability","release_year"
            ]].head(10))

            print("\nInterpretation hints:")
            print("- 'sim' = cosine similarity score (higher = closer to session mood).")
            print("- Compare each recommended song's tempo/valence/energy/danceability to the last-3 songs.")
            print("- This is how real session-based recommenders adapt to mood in real time.")


Last 3 listens (with audio features):


,ts,song_id,played_ms,skipped,tempo,valence,energy,danceability,release_year
0,2018-07-15 13:30:00+00:00,t_f35eb373-15d1-4faa-9ca0-d0da975deebb,172800,True,100.007,0.333133,0.428397,0.879681,2017
1,2018-07-15 13:30:00+00:00,t_b20489b0-ea5e-4071-afa7-91b55af6b577,293434,True,144.981,0.617478,0.696240,0.819664,2017
2,2018-07-15 13:30:00+00:00,t_48995243-c1c2-488d-9283-20c4621e2999,245226,True,92.326,0.316800,0.477740,0.544552,2016



Last-3 song IDs: ['t_f35eb373-15d1-4faa-9ca0-d0da975deebb', 't_b20489b0-ea5e-4071-afa7-91b55af6b577', 't_48995243-c1c2-488d-9283-20c4621e2999']

Session embedding (scaled feature vector):
[[-0.30773459 -0.2753139  -0.52024028  0.81181195]]

Top 10 recommendations:


KeyError: "['tempo', 'valence', 'energy', 'danceability'] not in index"

In [23]:
# Run this to show exactly which columns exist in songs
print("songs columns:", songs.columns.tolist())
display(songs.head(3))


songs columns: ['song_id', 'release_year', 'duration_ms', 'tempo', 'valence', 'energy', 'danceability']


,song_id,release_year,duration_ms,tempo,valence,energy,danceability
0,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,2018,180,134.025,0.152255,0.553465,0.653119
1,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,2018,236,130.038,0.337152,0.726828,0.877394
2,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,2018,231,145.028,0.373862,0.563009,0.679719


In [26]:
# --- Clean and robust recommender setup ---

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

FEATURE_COLS = ["tempo", "valence", "energy", "danceability"]
LAST_N = 3
TOP_K = 10

# 1) Make sure feature columns exist; if not, create them
for col in FEATURE_COLS:
    if col not in songs.columns:
        songs[col] = np.nan

# 2) Coerce to numeric (very important!)
for col in FEATURE_COLS:
    songs[col] = pd.to_numeric(songs[col], errors="coerce")

# 3) Fill remaining NaNs with column means
songs_clean = songs.copy()
songs_clean[FEATURE_COLS] = songs_clean[FEATURE_COLS].fillna(songs_clean[FEATURE_COLS].mean())

# 4) Build feature matrix
songs_feat = songs_clean[["song_id"] + FEATURE_COLS].copy()
songs_feat_matrix = songs_feat[FEATURE_COLS].values.astype(float)

# 5) Scale features
scaler = StandardScaler()
songs_feat_scaled = scaler.fit_transform(songs_feat_matrix)

# 6) Map song_id → index in feature matrix
id_to_idx = {sid: i for i, sid in enumerate(songs_feat["song_id"].values)}

# --- Helper: get last N songs in a session ---
def get_last_n_song_ids(user_id, session_id, n=LAST_N):
    sess = df[(df.user_id == user_id) & (df.session_id == session_id)].sort_values("ts")
    return list(sess["song_id"].tail(n))

# --- Helper: build session embedding ---
def session_embedding_from_ids(song_ids):
    vectors = []
    for sid in song_ids:
        idx = id_to_idx.get(sid)
        if idx is not None:
            vectors.append(songs_feat_scaled[idx])
    if len(vectors) == 0:
        return None
    return np.mean(vectors, axis=0).reshape(1, -1)

# --- Main recommender ---
def recommend_for_session(user_id, session_id, top_k=TOP_K, last_n=LAST_N):
    last_songs = get_last_n_song_ids(user_id, session_id, n=last_n)
    if not last_songs:
        return None

    session_vec = session_embedding_from_ids(last_songs)
    if session_vec is None:
        return None

    sims = cosine_similarity(session_vec, songs_feat_scaled).flatten()

    recs = songs_feat.copy()
    recs["sim"] = sims

    # remove songs already played in the session
    played = set(df[(df.user_id == user_id) & (df.session_id == session_id)]["song_id"])
    recs = recs[~recs["song_id"].isin(played)]

    return recs.sort_values("sim", ascending=False).head(top_k)


In [28]:
# Debug: inspect the recommendation result and songs table
print("recs columns:", recs.columns.tolist())
print("recs sample rows:")
display(recs.head(5))

print("\nsongs columns:", songs.columns.tolist())
print("songs sample rows:")
display(songs.head(5))

# Also show the merged 'out' columns and a small sample to see what happened
out_try = recs.merge(songs, on="song_id", how="left")
print("\nout_try columns after merge:", out_try.columns.tolist())
display(out_try.head(5))


recs columns: ['song_id', 'tempo', 'valence', 'energy', 'danceability', 'sim']
recs sample rows:


,song_id,tempo,valence,energy,danceability,sim
32201,t_0405bd48-851b-47c9-816e-0395a9e6583b,100.874,0.336983,0.399529,0.893032,0.999263
22530,t_bf2aa0e4-edb3-4146-a50a-fb6e34d1c56f,116.995,0.463943,0.593017,0.674371,0.998727
13804,t_585216d3-3825-40fc-b4bc-6f885d4cb70b,107.997,0.390104,0.455223,0.846787,0.998521
15616,t_e2c5125e-c96e-49c4-9e71-6cf6405e501b,106.997,0.385472,0.501576,0.797315,0.998076
26904,t_a8b1c27a-f045-44bc-afb2-a872188456f4,111.989,0.435094,0.536507,0.756451,0.997831



songs columns: ['song_id', 'release_year', 'duration_ms', 'tempo', 'valence', 'energy', 'danceability']
songs sample rows:


,song_id,release_year,duration_ms,tempo,valence,energy,danceability
0,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,2018,180,134.025,0.152255,0.553465,0.653119
1,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,2018,236,130.038,0.337152,0.726828,0.877394
2,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,2018,231,145.028,0.373862,0.563009,0.679719
3,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,2018,169,111.982,0.649420,0.529484,0.864881
4,t_64f3743c-f624-46bb-a579-0f3f9a07a123,2018,210,147.031,0.652921,0.650057,0.857778



out_try columns after merge: ['song_id', 'tempo_x', 'valence_x', 'energy_x', 'danceability_x', 'sim', 'release_year', 'duration_ms', 'tempo_y', 'valence_y', 'energy_y', 'danceability_y']


,song_id,tempo_x,valence_x,energy_x,danceability_x,sim,release_year,duration_ms,tempo_y,valence_y,energy_y,danceability_y
0,t_0405bd48-851b-47c9-816e-0395a9e6583b,100.874,0.336983,0.399529,0.893032,0.999263,2016,200,100.874,0.336983,0.399529,0.893032
1,t_bf2aa0e4-edb3-4146-a50a-fb6e34d1c56f,116.995,0.463943,0.593017,0.674371,0.998727,2016,233,116.995,0.463943,0.593017,0.674371
2,t_585216d3-3825-40fc-b4bc-6f885d4cb70b,107.997,0.390104,0.455223,0.846787,0.998521,2016,200,107.997,0.390104,0.455223,0.846787
3,t_e2c5125e-c96e-49c4-9e71-6cf6405e501b,106.997,0.385472,0.501576,0.797315,0.998076,2003,236,106.997,0.385472,0.501576,0.797315
4,t_a8b1c27a-f045-44bc-afb2-a872188456f4,111.989,0.435094,0.536507,0.756451,0.997831,2018,197,111.989,0.435094,0.536507,0.756451


In [29]:
# Cleanly display top recommendations with features + metadata
# 'recs' already has tempo/valence/energy/danceability and sim
# 'songs' has release_year and duration_ms which we want too

out_clean = recs.merge(
    songs[["song_id", "release_year", "duration_ms"]],  # only metadata to avoid duplicates
    on="song_id",
    how="left"
)

# show columns we want (features come from recs)
display(out_clean[[
    "song_id", "sim", "tempo", "valence", "energy", "danceability",
    "release_year", "duration_ms"
]].head(10))


,song_id,sim,tempo,valence,energy,danceability,release_year,duration_ms
0,t_0405bd48-851b-47c9-816e-0395a9e6583b,0.999263,100.874,0.336983,0.399529,0.893032,2016,200
1,t_bf2aa0e4-edb3-4146-a50a-fb6e34d1c56f,0.998727,116.995,0.463943,0.593017,0.674371,2016,233
2,t_585216d3-3825-40fc-b4bc-6f885d4cb70b,0.998521,107.997,0.390104,0.455223,0.846787,2016,200
3,t_e2c5125e-c96e-49c4-9e71-6cf6405e501b,0.998076,106.997,0.385472,0.501576,0.797315,2003,236
4,t_a8b1c27a-f045-44bc-afb2-a872188456f4,0.997831,111.989,0.435094,0.536507,0.756451,2018,197
5,t_5f2e6e34-b9ff-47e7-b5a4-38ab7f70de7b,0.997708,105.050,0.371602,0.486971,0.825306,2018,218
6,t_dbe34bef-b308-434e-b3b8-2fb322a16336,0.997399,100.087,0.360374,0.354053,0.947412,1990,302
7,t_211617e6-8574-4d31-be26-4cef68379220,0.997227,103.010,0.383230,0.464793,0.867140,2016,207
8,t_e6e757ae-6680-4e09-a59a-b19525bc28ee,0.997105,110.237,0.424136,0.517859,0.749777,2017,249
9,t_1f902d21-6120-49a5-9e88-710743681cc5,0.997069,118.005,0.456832,0.600828,0.664222,2013,294
